In [1]:
import torch
import torchvision
import torch.nn  as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision import models
import os
import glob
import shutil
import random
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

In [3]:
from mytorch import utils
from mytorch import myResNet50
import importlib
importlib.reload(utils)
importlib.reload(myResNet50)

<module 'mytorch.myResNet50' from '/Users/water_mbp/workspace/Labo/floc/debuet/mytorch/myResNet50.py'>

In [ ]:
train_csv = pd.read_csv("./csv/train.csv")
test_csv = pd.read_csv("./csv/test.csv")

data_transforms = transforms.Compose([
    transforms.ToTensor(),
])

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    classes = ['1', '2', '3']

    def __init__(self, root, transform=None):
        '''
        root: 画像が保存されているパス
        transform: transform
        '''
        self.transform = transform
        self.images = []
        self.labels = []
        root_path = root
        files = glob.glob(os.path.join(root_path, '0/*.jpg'))
        for path in files:
            self.images.append(path)
            self.labels.append(0)
        files = glob.glob(os.path.join(root_path, '1/*.jpg'))
        for path in files:
            self.images.append(path)
            self.labels.append(1)
        files = glob.glob(os.path.join(root_path, '2/*.jpg'))
        for path in files:
            self.images.append(path)
            self.labels.append(2)
    
    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]
        with open(image, 'rb') as f:
            image = Image.open(f)
            image = image.convert('RGB')
        if self.transform is not None:
            image = self.transform(image)
        return image, label
    
    def __len__(self):
        return len(self.images)

In [ ]:
train_root = "./dataset/train_dataset/"
vali_root = "./dataset/vali_dataset/"
test_root = "./dataset/test_dataset/"

custom_train_dataset = CustomDataset(train_root, data_transforms)
custom_vali_dataset = CustomDataset(vali_root, data_transforms)
custom_test_dataset = CustomDataset(test_root, data_transforms)

train_loader = torch.utils.data.DataLoader(
    dataset=custom_train_dataset,
    batch_size=32,
    shuffle=True
)
vali_loader = torch.utils.data.DataLoader(
    dataset=custom_vali_dataset,
    batch_size=32,
    shuffle=False
)
test_loader = torch.utils.data.DataLoader(
    dataset=custom_test_dataset,
    batch_size=32,
    shuffle=False
)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = myResNet50.MyResNet(myResNet50.MyBottleneck, [3, 4, 6, 3])
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

num_epoch = 2

train_loss_list = []
train_acc_list = []
val_loss_list = []
val_acc_list = []

for epoch in range(num_epoch):
    #initializer
    train_loss = 0
    train_acc = 0
    val_loss = 0
    val_acc = 0
    #train
    net.train()
    # mini_bach毎に読み込む
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(images)
        loss = criterion(outputs, labels)
        train_loss += loss.item()
        train_acc += (outputs.max(1)[1] == labels).sum().item()
        loss.backward()
        optimizer.step()
    avg_train_loss = train_loss / len(train_loader.dataset)
    avg_train_acc = train_acc / len(train_loader.dataset)
    #validation
    net.eval()
    with torch.no_grad():
        for images, labels in enumerate(vali_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            val_acc += (outputs.max(1)[1] == labels).sum().item()
        avg_val_loss = val_loss / len(vali_loader.dataset)
        avg_val_acc = val_acc / len(vali_loader.dataset)
    print('Epoch [{}/{}], Loss: {loss:.4f}, val_loss: {val_loss:.4f}, val_acc: {val_acc:.4f}'.format(
        epoch+1, 
        num_epoch, 
        loss=avg_train_loss, 
        val_loss=avg_val_loss,
        val_acc=avg_val_acc
        ))
    train_loss_list.append(avg_train_loss)
    train_acc_list.append(avg_train_acc)
    val_loss_list.append(avg_val_loss)
    val_acc_list.append(avg_val_acc)

In [ ]:
utils.utils.plot_epoch_to_loss_acc(num_epoch, train_loss_list, val_loss_list, train_acc_list, val_acc_list)

In [ ]:
outputs = utils.utils.test(net, test_loader, "./outcome/heatmap.png")
utils.utils.split_correct_image_csv(
    test_csv, 
    "./outcome/outcome.csv", 
    "./outcome/correctImages/", 
    "./outcome/incorrectImages/", 
    custom_test_dataset, outputs
    )